In [ ]:
import csv
import importlib
import itertools
import operator
import os
import random
import time
from math import e, exp, log, pi, sqrt
import os
import psutil

import keras.layers as layers
import copy
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
from numba import guvectorize, jit, njit, prange, vectorize
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import tensorflow as tf
from os.path import *
import glob
import cnn_builder as cbuild
import cnn_runner as crun
import config
import dr_methods as drm
import feature_interpretation as cnna
import niftiutils.helper_fxns as hf
import niftiutils.private as prv
import niftiutils.transforms as tr
import niftiutils.visualization as vis
import voi_methods as vm
import feature_influence as finf

%matplotlib inline
np.set_printoptions(3)

In [2]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(drm)
importlib.reload(cbuild)
importlib.reload(cnna)
importlib.reload(crun)
C = config.Config("etiology")
T = config.Hyperparams()

### backup

In [15]:
model_num = 21
model_path = join(C.model_dir, "model_reader_new%d.hdf5" % model_num)

Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
orig_data_dict, num_samples = cbuild._collect_unaug_data()

features_by_cls, feat_count = cnna.collect_features()
feat_count.pop("homogeneous texture")
#feat_count.pop("central scar")
all_features = sorted(list(feat_count.keys()))
cls_features = {f: [c for c in C.cls_names if f in features_by_cls[c]] for f in all_features}

Z_features = cnna.get_annotated_files(features_by_cls)
Z_features.pop("homogeneous texture")
#Z_features.pop("central scar")

num_features = len(all_features)

voi_df = drm.get_voi_dfs()[0]
M = keras.models.load_model(model_path)

M.layers[5].get_weights()[1]

In [16]:
def memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    print('Memory use:', py.memory_info()[0]/2.**30)

In [31]:
model_num = 26
model_path = join(C.model_dir, "model_reader_new%d.hdf5" % model_num)
inf_xls_path = 'D:\\feature_analysis\\influence%d.xlsx' % model_num

df = pd.DataFrame(columns=["true_cls", "pred_cls"] + all_features)
#df = pd.read_excel(inf_xls_path, index_col=0)

In [18]:
Z_full = np.concatenate([orig_data_dict[cls][-1] for cls in C.cls_names],0)

all_imgs = []
all_cls = []

for lesion_id in Z_full:
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    all_imgs.append(np.expand_dims(img,0))
    all_cls.append(C.cls_names.index(cls))

In [32]:
importlib.reload(finf)
del M, IA
K.clear_session()
M = keras.models.load_model(model_path)
IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)

In [ ]:
t = time.time()
for lesion_id in Z_reader:
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    img = np.expand_dims(img,0)
    pred_cls = M.predict(img)[0]
    pred_cls = C.cls_names[list(pred_cls).index(pred_cls.max())]
        
    print(lesion_id)
    
    g_test = IA.get_grad(lesion_id, pred_cls=pred_cls)
    s_test = IA.get_stest(g_test)

    del M, IA
    K.clear_session()
    M = keras.models.load_model(model_path)
    IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)

    I = {}
    for f in Z_features:
        I[f] = IA.get_avg_influence(Z_features[f], s_test)

    del M, IA
    K.clear_session()
    M = keras.models.load_model(model_path)
    IA = finf.InfluenceAnalyzer(M, voi_df, all_imgs, all_cls)
    
    df.loc[lesion_id] = [cls, pred_cls] + list(I[f] for f in df.columns[2:])

    print(time.time() - t)
    memory()

In [36]:
inf_xls_path

'D:\\feature_analysis\\influence26.xlsx'

In [37]:
df.to_excel(inf_xls_path)

In [ ]:
cls = voi_df.loc[lesion_id]["cls"]
x_test = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
vis.draw_slices(x_test)

Memory leak test
for _ in range(10):
    memory()
    g_test = IA.get_grad(lesion_id)
memory()

### extra

In [2]:
def perturb_weights(W_true, t):
    eps = 1e-5
    W = copy.deepcopy(W_true)
    t_ix = 0
    
    for w_ix in range(len(W)):
        W[w_ix] += eps * np.reshape(t[t_ix:t_ix+W[w_ix].size], W[w_ix].shape)
        t_ix += W[w_ix].size
        
    return W

In [3]:
def get_grad(lesion_id, perturb_W=None):
    C = config.Config()
    
    cls = voi_df.loc[lesion_id]["cls"]
    img = np.load(join(C.orig_dir, cls, lesion_id+".npy"))
    img = np.expand_dims(img,0)

    y_true = np_utils.to_categorical(C.cls_names.index(cls), 6)
    loss = K.categorical_crossentropy(y_true, M.output)
    
    with tf.device('/gpu:0'):
        g = K.gradients(loss, M.trainable_weights)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        g_i = sess.run(g, feed_dict={M.input:img, K.learning_phase():0})
        if perturb_W is not None:
            feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
            g_i_plus = sess.run(g, feed_dict={**feed_dict_plus, M.input:img, K.learning_phase():0})

    g_i = np.concatenate([x.flatten() for x in g_i], 0)
    
    if perturb_W is not None:
        g_i_plus = np.concatenate([x.flatten() for x in g_i_plus], 0)
        return g_i, g_i_plus
    
    return g_i

In [4]:
def get_grads(Z_sample):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(Z_sample)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
    if verbose:
        print(time.time()-t)
    
    return Ht / len(Z_sample)

In [24]:
def get_HVP(perturb_W, Z_sample, verbose=True):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    feed_dict_plus = {M.trainable_weights[i]:perturb_W[i] for i in range(len(perturb_W))}
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(Z_sample)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i_plus = sess.run(g[classes[ix]], feed_dict={**feed_dict_plus, M.input:imgs[ix], K.learning_phase():0})
    
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
            g_i_plus = np.concatenate([x.flatten() for x in g_i_plus], 0)

            Ht += (g_i_plus - g_i)/eps
    if verbose:
        print(time.time()-t)
    
    return Ht / len(Z_sample)

In [23]:
t_k = np.zeros(g_test.shape)
r_k = g_test#-Ht
p_k = r_k

In [ ]:
num_iters = 15
#phi_hist = []
t = time.time()
for _ in range(num_iters):
    #Z_sample = random.sample(list(Z_full), 15)
    W_new = perturb_weights(W, p_k)
    Hp = get_HVP(W_new, Z_full)

    alpha = np.dot(r_k, r_k) / np.dot(p_k, Hp)
    t_k += alpha * p_k
    r_k2 = r_k - alpha * Hp
    #phi_hist = .5*np.dot(t_k, Ht) - g_test

    beta = np.dot(r_k2, r_k2) / np.dot(r_k, r_k)
    r_k = r_k2
    p_k = r_k + beta*p_k
print(time.time()-t)

In [33]:
del p_k

In [36]:
def get_grads(imgs, classes, s_test, verbose=True):
    eps = 1e-5
    Ht = np.zeros(g_test.shape)
    losses = [K.categorical_crossentropy(y_true, M.output) for y_true in \
              [np_utils.to_categorical(i, 6) for i in range(6)]]
        
    I = 0
    t = time.time()
    with tf.device('/gpu:0'):
        g = [K.gradients(loss, M.trainable_weights) for loss in losses]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for ix in range(len(classes)):
            g_i = sess.run(g[classes[ix]], feed_dict={M.input:imgs[ix], K.learning_phase():0})
            g_i = np.concatenate([x.flatten() for x in g_i], 0)
            I -= np.dot(g_i, s_test)
    if verbose:
        print(time.time()-t)
    
    return I / len(classes)

### others

In [47]:
df_0 = hf.pickle_load(join(C.base_dir, "data", "DFs4.bin"))[0]

In [41]:
data_dir = r"C:\Users\Clinton\Documents\voi-classifier\data"
answer_key = join(data_dir, "ground_truth.xlsx")
answer_key = pd.read_excel(answer_key, index_col=0)

In [46]:
answer_key.columns

Index(['true_cls', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'ignore_1', 'ignore_2', 'BLANK', 'Unnamed: 9', 'Class', 'Num features'],
      dtype='object')

In [53]:
for z_test, row in answer_key.iterrows():
    cls = df_0.loc[z_test, "pred_cls"]
    feats = row[[f for f in answer_key.columns if f.startswith('feat')]].dropna().values
    break

In [58]:
feats.dropna().values

array(['nodular growth', 'heterogeneous lesion',
       'progressive enhancement'], dtype=object)